In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa
import seaborn as sns

In [ ]:
EXPORT = 0
CO2L = 0.10 #
CLUSTERS = 4
H = 144
LL = "3.0"
DISCOUNT_RATE = 0.15

RUN = "newmain_144H_ws" #"dyntrans_full_3H_ws"
CATEGORY = "postnetworks"
if CO2L == 2.0:
    SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L:.1f}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"
else:
    SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L:.2f}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
OUTPUT = PREFIX + "results"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/graphics/demand/{SCENARIO}"
OUTPUT_RAW = f"{OUTPUT}/{RUN}/graphics/demand/"

In [ ]:
n = pypsa.Network(INPUT)

### Check balance of "co2 atmosphere"

In [ ]:
energy_balance = n.statistics.energy_balance(aggregate_bus=False, aggregate_time=False)

In [ ]:
co2_balance = energy_balance.loc[:,:,:, "co2 atmosphere"].sum(axis=1) * n.snapshot_weightings.generators[0] / 1e6
co2_balance = co2_balance.round(2)

In [ ]:
co2_balance[co2_balance > 0.01] 

In [ ]:
co2_balance[co2_balance < -0.01] 

### Emissions of generators

In [ ]:
emissions_generator = n.generators_t.p / n.generators.efficiency * n.generators.carrier.map(n.carriers.co2_emissions) # t/h
total_em_generator = n.snapshot_weightings.generators @ emissions_generator.sum(axis=1).div(1e6) # MtCO2

In [ ]:
total_em_generator

### Total emissions

In [ ]:
abs(co2_balance.loc[:,"co2",:][0])

In [ ]:
(abs(total_em_generator) + abs(co2_balance.loc[:,"co2",:][0])).round(2) # MtCO2

### Compare with Co2L

In [ ]:
n.global_constraints.loc["CO2Limit"].constant /1e6 # in Mt

### Automized analysis across co2ls

In [ ]:
def get_carbon_stats(limits):

    carbon_stats = pd.DataFrame()


    for co2l in limits:

        if co2l == 2.0:
            SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{co2l:.1f}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"
        else:
            SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{co2l:.2f}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

        INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"

        n = pypsa.Network(INPUT)

        # Calculate co2 emissions from links
        energy_balance = n.statistics.energy_balance(aggregate_bus=False, aggregate_time=False)
        co2_balance = energy_balance.loc[:,:,:, "co2 atmosphere"].sum(axis=1) * n.snapshot_weightings.generators[0] / 1e6
        co2_link = abs(co2_balance.loc[:,"co2",:][0])

        # Calculate co2 emissions from generators
        emissions_generator = n.generators_t.p / n.generators.efficiency * n.generators.carrier.map(n.carriers.co2_emissions) # t/h
        co2_generator = n.snapshot_weightings.generators @ emissions_generator.sum(axis=1).div(1e6) # MtCO2

        # Get constraint
        constraint = n.global_constraints.loc["CO2Limit"].constant /1e6 # in Mt

        carbon_stats.loc[co2l, "co2_link"] = co2_link.round(2)
        carbon_stats.loc[co2l, "co2_generator"] = co2_generator.round(2)
        carbon_stats.loc[co2l, "co2_total"] = co2_link.round(2) + co2_generator.round(2)
        carbon_stats.loc[co2l, "constraint"] = constraint.round(2)

    return carbon_stats

In [ ]:
limits = [2.0, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00]
carbon_stats = get_carbon_stats(limits)

In [ ]:
carbon_stats

In [ ]:
# Plot the carbon stats
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6))
carbon_stats[["co2_link", "co2_generator", "co2_total"]].iloc[:,:].plot(ax=ax)
carbon_stats["constraint"].iloc[:].plot(ax=ax, style="k--", label="constraint")
ax.set_ylabel("Emissions in MtCO2")
ax.set_xlabel("CO2 limit in MtCO2")
ax.set_title("CO2 emissions in 2030")
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
ax.grid()
ax.invert_xaxis()
fig.tight_layout()